In [8]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time

In [9]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [10]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. " 
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [11]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [32]:
BATCH_SIZE = 256
transoforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])
train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)



# AlexNet

In [12]:
net = nn.Sequential(
    nn.Conv2d(1, 96, kernel_size=11, stride=4),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Conv2d(96, 256, kernel_size=5, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(3, stride=2),
    nn.Flatten(),# 3d tensro to 1d
    nn.Linear(6400, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)

In [11]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
              ReLU-2           [-1, 96, 54, 54]               0
         MaxPool2d-3           [-1, 96, 26, 26]               0
            Conv2d-4          [-1, 256, 26, 26]         614,656
              ReLU-5          [-1, 256, 26, 26]               0
         MaxPool2d-6          [-1, 256, 12, 12]               0
            Conv2d-7          [-1, 384, 12, 12]         885,120
              ReLU-8          [-1, 384, 12, 12]               0
            Conv2d-9          [-1, 384, 12, 12]       1,327,488
             ReLU-10          [-1, 384, 12, 12]               0
           Conv2d-11          [-1, 256, 12, 12]         884,992
             ReLU-12          [-1, 256, 12, 12]               0
        MaxPool2d-13            [-1, 256, 5, 5]               0
          Flatten-14                 [-

In [162]:
lr, num_epochs  = 0.001, 1
trainer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, trainer, num_epochs)

Step 0. time since epoch: 2.463. Train acc: 0.117. Train Loss: 2.302
Step 10. time since epoch: 29.936. Train acc: 0.114. Train Loss: 2.348
Step 20. time since epoch: 58.440. Train acc: 0.247. Train Loss: 2.142
Step 30. time since epoch: 87.069. Train acc: 0.372. Train Loss: 1.824
Step 40. time since epoch: 115.854. Train acc: 0.475. Train Loss: 1.551
Step 50. time since epoch: 144.508. Train acc: 0.549. Train Loss: 1.341
Step 60. time since epoch: 173.514. Train acc: 0.601. Train Loss: 1.189
Step 70. time since epoch: 202.803. Train acc: 0.641. Train Loss: 1.071
Step 80. time since epoch: 231.832. Train acc: 0.678. Train Loss: 0.965
Step 90. time since epoch: 261.006. Train acc: 0.707. Train Loss: 0.880
Step 100. time since epoch: 290.201. Train acc: 0.730. Train Loss: 0.811
Step 110. time since epoch: 319.686. Train acc: 0.749. Train Loss: 0.753
Step 120. time since epoch: 349.286. Train acc: 0.766. Train Loss: 0.703
Step 130. time since epoch: 378.594. Train acc: 0.781. Train Loss: 

# VGG

In [36]:

def vgg_block(num_convs, input_channels, num_channels):
    blk = nn.Sequential(nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1), nn.ReLU())
    for i in range(num_convs - 1):
        blk.add_module("conv{}".format(i), nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1))
        blk.add_module("relu{}".format(i), nn.ReLU())
    blk.add_module("pool", nn.MaxPool2d(2, stride=2))
    return blk

In [37]:

def vgg(conv_arch):
    net = nn.Sequential()

    for i, (num_convs, input_ch, num_channels) in enumerate(conv_arch):
        net.add_module("block{}".format(i), vgg_block(num_convs, input_ch, num_channels))

    
    classifier = nn.Sequential(
        nn.Flatten(),
        
        # nn.Linear(25088, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(6272, 4096), nn.ReLU(), nn.Dropout(0.5),

        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))

    net.add_module('classifier', classifier)
    return net

In [38]:
conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))

In [39]:
ratio = 4
small_conv_arch = [(v[0], max(v[1] // ratio, 1), v[2] // ratio) for v in conv_arch]
net = vgg(small_conv_arch)

In [40]:
small_conv_arch

[(1, 1, 16), (1, 16, 32), (2, 32, 64), (2, 64, 128), (2, 128, 128)]

In [41]:
224*224

50176

In [42]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             160
              ReLU-2         [-1, 16, 224, 224]               0
         MaxPool2d-3         [-1, 16, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]           4,640
              ReLU-5         [-1, 32, 112, 112]               0
         MaxPool2d-6           [-1, 32, 56, 56]               0
            Conv2d-7           [-1, 64, 56, 56]          18,496
              ReLU-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]          36,928
             ReLU-10           [-1, 64, 56, 56]               0
        MaxPool2d-11           [-1, 64, 28, 28]               0
           Conv2d-12          [-1, 128, 28, 28]          73,856
             ReLU-13          [-1, 128, 28, 28]               0
           Conv2d-14          [-1, 128,

In [43]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, trainer, num_epochs)

Step 0. time since epoch: 2.963. Train acc: 0.082. Train Loss: 2.303
Step 10. time since epoch: 34.919. Train acc: 0.103. Train Loss: 2.303
Step 20. time since epoch: 67.279. Train acc: 0.103. Train Loss: 2.302
Step 30. time since epoch: 99.903. Train acc: 0.107. Train Loss: 2.302
Step 40. time since epoch: 132.689. Train acc: 0.108. Train Loss: 2.302
Step 50. time since epoch: 165.912. Train acc: 0.123. Train Loss: 2.292
Step 60. time since epoch: 199.295. Train acc: 0.177. Train Loss: 2.172
Step 70. time since epoch: 232.103. Train acc: 0.254. Train Loss: 1.984
Step 80. time since epoch: 265.396. Train acc: 0.326. Train Loss: 1.801
Step 90. time since epoch: 298.995. Train acc: 0.389. Train Loss: 1.639
Step 100. time since epoch: 331.987. Train acc: 0.441. Train Loss: 1.502
Step 110. time since epoch: 365.464. Train acc: 0.486. Train Loss: 1.384
Step 120. time since epoch: 399.007. Train acc: 0.523. Train Loss: 1.287
Step 130. time since epoch: 432.726. Train acc: 0.556. Train Loss: 

# NiN

In [175]:
def nin_block(input_channels, num_channels, kernel_size, strides, padding):
    blk = nn.Sequential(
          nn.Conv2d(input_channels, num_channels, kernel_size, strides, padding),
          nn.ReLU(),
          nn.Conv2d(num_channels, num_channels, kernel_size=1),
          nn.ReLU(),
          nn.Conv2d(num_channels, num_channels, kernel_size=1),
          nn.ReLU()
    )
    return blk

In [176]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AvgPool2d(5),
    nn.Flatten()
)

In [177]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          11,712
              ReLU-2           [-1, 96, 54, 54]               0
            Conv2d-3           [-1, 96, 54, 54]           9,312
              ReLU-4           [-1, 96, 54, 54]               0
            Conv2d-5           [-1, 96, 54, 54]           9,312
              ReLU-6           [-1, 96, 54, 54]               0
         MaxPool2d-7           [-1, 96, 26, 26]               0
            Conv2d-8          [-1, 256, 26, 26]         614,656
              ReLU-9          [-1, 256, 26, 26]               0
           Conv2d-10          [-1, 256, 26, 26]          65,792
             ReLU-11          [-1, 256, 26, 26]               0
           Conv2d-12          [-1, 256, 26, 26]          65,792
             ReLU-13          [-1, 256, 26, 26]               0
        MaxPool2d-14          [-1, 256,

In [ ]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, trainer, num_epochs)

# GoogleNet

In [178]:
nn.Module

torch.nn.modules.module.Module

In [179]:
class Inception(nn.Module):
    def __init__(self, ic, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        self.p1_1 = nn.Sequential(nn.Conv2d(ic, c1, kernel_size=1), nn.ReLU())
        self.p2_1 = nn.Sequential(nn.Conv2d(ic, c2[0], kernel_size=1), nn.ReLU())
        self.p2_2 = nn.Sequential(nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1), nn.ReLU())
        self.p3_1 = nn.Sequential(nn.Conv2d(ic, c3[0], kernel_size=1), nn.ReLU())
        self.p3_2 = nn.Sequential(nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2), nn.ReLU())
        self.p4_1 = nn.Sequential(nn.MaxPool2d(3, stride=1, padding=1))
        self.p4_2 = nn.Sequential(nn.Conv2d(ic, c4, kernel_size=1), nn.ReLU())

    def forward(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        # Concatenate the outputs on the channel dimension.
        return torch.cat((p1, p2, p3, p4), dim=1)

In [180]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), nn.ReLU(),
       nn.MaxPool2d(3, stride=2, padding=1))

In [181]:

b2 = nn.Sequential(
       nn.Conv2d(64, 64, kernel_size=1),
       nn.Conv2d(64, 192, kernel_size=3, padding=1),
       nn.MaxPool2d(3, stride=2, padding=1))

In [182]:

b3 = nn.Sequential(
       Inception(192, 64, (96, 128), (16, 32), 32),
       Inception(256, 128, (128, 192), (32, 96), 64),
       nn.MaxPool2d(3, stride=2, padding=1))

In [183]:

b4 = nn.Sequential(
       Inception(480, 192, (96, 208), (16, 48), 64),
       Inception(512, 160, (112, 224), (24, 64), 64),
       Inception(512, 128, (128, 256), (24, 64), 64),
       Inception(512, 112, (144, 288), (32, 64), 64),
       Inception(528, 256, (160, 320), (32, 128), 128),
       nn.MaxPool2d(3, stride=2, padding=1))

In [184]:

b5 = nn.Sequential(
       Inception(832, 256, (160, 320), (32, 128), 128),
       Inception(832, 384, (192, 384), (48, 128), 128),
       nn.AvgPool2d(7))

In [185]:
net = nn.Sequential(b1, b2, b3, b4, b5, nn.Flatten(), nn.Linear(1024, 10))

In [186]:
summary(net.to(device), input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           3,200
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]           4,160
            Conv2d-5          [-1, 192, 56, 56]         110,784
         MaxPool2d-6          [-1, 192, 28, 28]               0
            Conv2d-7           [-1, 64, 28, 28]          12,352
              ReLU-8           [-1, 64, 28, 28]               0
            Conv2d-9           [-1, 96, 28, 28]          18,528
             ReLU-10           [-1, 96, 28, 28]               0
           Conv2d-11          [-1, 128, 28, 28]         110,720
             ReLU-12          [-1, 128, 28, 28]               0
           Conv2d-13           [-1, 16, 28, 28]           3,088
             ReLU-14           [-1, 16,

In [206]:
lr, num_epochs = 0.001, 1
trainer = torch.optim.Adam(net.parameters(), lr=lr)
train(net, train_iter, test_iter, trainer, num_epochs)

RuntimeError: Given groups=1, weight of size [64, 1, 7, 7], expected input[256, 3, 224, 224] to have 1 channels, but got 3 channels instead

# DenseNet

In [ ]:
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = tv.models.densenet161(pretrained=True)

In [ ]:
model

In [ ]:

# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.classifier

In [ ]:
model.classifier = nn.Linear(in_features=2208, out_features=10)

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
train(model, train_iter, test_iter, trainer, 1)

# DenseNet

In [ ]:
model = tv.models.densenet161(pretrained=True)

In [ ]:

# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False
model.classifier


In [ ]:
model.classifier = nn.Linear(in_features=2208, out_features=10)

In [ ]:

print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:

train(model, train_iter, test_iter, trainer, 1)

In [205]:
import numpy as np

# Задаем входные данные и целевые значения
X = np.array([[0, 1, 0],
              [1, 1, 1],
              [1, 0, 0],
              [0, 1, 1]])
y = np.array([[0],
              [1],
              [1],
              [0]])

# Задаем параметры сети
input_neurons = X.shape[1]  # Количество входных нейронов
hidden1_neurons = 8 
hidden2_neurons = 4  # Количество нейронов во втором скрытом слое
output_neurons = 1  # Количество выходных нейронов
epochs = 10000  # Количество эпох обучения
learning_rate = 0.1  # Скорость обучения

# Инициализируем веса сети случайными значениями
w1 = np.random.uniform(size=(input_neurons, hidden1_neurons))
w2 = np.random.uniform(size=(hidden1_neurons, hidden2_neurons))
w3 = np.random.uniform(size=(hidden2_neurons, output_neurons))

# Функция активации (sigmoid)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Функция потерь (MSE)
def mse_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Процесс обучения
for i in range(epochs):
    # Прямое распространение сигнала
    hidden1_layer = sigmoid(np.dot(X, w1))
    hidden2_layer = sigmoid(np.dot(hidden1_layer, w2))
    output_layer = sigmoid(np.dot(hidden2_layer, w3))

    # Вычисление ошибки
    error = np.mean(np.square(y - output_layer))
    error = output_layer - y 

    # Обновление весов сети с помощью обратного распространения ошибки
    d_output = error * output_layer * (1 - output_layer)
    d_hidden2 = hidden2_layer * (1 - hidden2_layer) * np.dot(d_output, w3.T)
    d_hidden1 = hidden1_layer * (1 - hidden1_layer) * np.dot(d_hidden2, w2.T)
    w3 -= learning_rate * np.dot(hidden2_layer.T, d_output)
    w2 -= learning_rate * np.dot(hidden1_layer.T, d_hidden2)
    w1 -= learning_rate * np.dot(X.T, d_hidden1)

    # Выводим текущее значение потерь
    if i % 1000 == 0:
        loss = mse_loss(y, output_layer)
        print(f"Epoch {i}, Loss: {loss}")

# Тестирование обученной сети
test_input = np.array([[0, 0, 1],
                       [1, 0, 1]])
hidden1_layer = sigmoid(np.dot(test_input, w1))
hidden2_layer = sigmoid(np.dot(hidden1_layer, w2))
test_output = sigmoid(np.dot(hidden2_layer, w3))
print(test_output)


Epoch 0, Loss: 0.3620720938568087
Epoch 1000, Loss: 0.24818488189413446
Epoch 2000, Loss: 0.13513528120338808
Epoch 3000, Loss: 0.004166378770697759
Epoch 4000, Loss: 0.001633221852659256
Epoch 5000, Loss: 0.0009742008772452404
Epoch 6000, Loss: 0.0006828323229207705
Epoch 7000, Loss: 0.0005212061530006792
Epoch 8000, Loss: 0.00041933929578465287
Epoch 9000, Loss: 0.0003496366610936228
[[0.45908929]
 [0.99055395]]


In [1]:
import numpy as np
np.random.seed(seed=1)
def gradient_descent(X, y, learning_rate=0.01, epochs=10000):
    # инициализация случайных весов
    weights = np.random.randn(X.shape[1])
    print(weights)
    # цикл обучения
    for epoch in range(epochs):
        # вычисление предсказаний
        y_pred = X.dot(weights)
        # вычисление ошибки
        error = ( y- y_pred  )
        # вычисление градиента
        gradient = X.T.dot(error) / X.shape[0]
         # обновление весов
        weights -= learning_rate * 2*gradient
    return weights

# пример использования
X = np.array([[1, 2], [3, 4], [5, 6]])
w = np.array([1,2])
y = X.dot(w)

weights = gradient_descent(X, y)
print(weights)


[ 1.62434536 -0.61175641]
[-inf -inf]


In [58]:
X.T.dot(error)

ValueError: shapes (2,3) and (4,1) not aligned: 3 (dim 1) != 4 (dim 0)

In [1]:
len("204805805314797448912")

21